# 2. Topic Modeling - Latent Dirchlet Allocation (LDA) 

In [1]:
# format code
# pip install nb_black
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pickle
import csv
import pandas as pd
import numpy as np
import scipy.sparse
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
from gensim import matutils
from gensim import models
from gensim import corpora
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

<IPython.core.display.Javascript object>

In [3]:
stop_words = set(stopwords.words("german"))
stop_words_add = [
    "mal",
    "mehr",
    "ja",
    "schon",
    "gibt",
    "geht",
    "hast",
    "einfach",
    "ganz",
    "macht",
    "immer",
    "tun",
    "viele",
    "wer",
    "sagen",
    "wäre",
    "genau",
    "dafür",
    "natürlich",
    "seit",
    "wurde",
    "eigentlich",
    "kommt",
    "gesagt",
    "sagt",
    "nie",
    "sehen",
    "deren",
    "versuchen",
    "empfehlen",
    "müssen",
    "kurz",
    "wenig",
    "erste",
    "klare",
    "gar",
    "grad",
    "wohl",
    "oft",
    "ha",
    "schaffen",
    "daher",
    "schreibt",
    "ständig",
    "völlig",
    "verdient",
    "worden",
    "solange",
    "könnt",
    "mann",
    "zeigt",
    "später",
    "erste",
    "iwelche",
    "wen",
    "eigenem",
    "gründen",
    "ups",
    "irgendjemand",
    "wuerde",
    "gründen",
]
sw = set(list(stop_words) + stop_words_add)

<IPython.core.display.Javascript object>

## 1. Topic Modeling on single comment corpus

### skip to load, if already stored

In [ ]:
dtm = pd.read_pickle("../data/pickle/dtm.pkl")
dtm

In [10]:
tdm = dtm.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,229627,229628,229629,229630,229631,229632,229633,229634,229635,229636
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [12]:
cv = pickle.load(open("../data/pickle/cv.pkl", "rb"))
print(len(cv.vocabulary_))

22383


<IPython.core.display.Javascript object>

In [13]:
rows_to_drop = []
for word in list(sw):
    if word in tdm.index:
        rows_to_drop.append(word)
        cv.vocabulary_.pop(word)

print(len(rows_to_drop))

55


<IPython.core.display.Javascript object>

In [14]:
print(len(cv.vocabulary_))

22328


<IPython.core.display.Javascript object>

In [15]:
tdm.drop(index=rows_to_drop, inplace=True)

<IPython.core.display.Javascript object>

In [16]:
tdm

,0,1,2,3,4,5,6,7,8,9,...,229627,229628,229629,229630,229631,229632,229633,229634,229635,229636
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übrigens,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übriges,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [17]:
pickle.dump(cv, open("../data/pickle/cv_sw_add.pkl", "wb"))
tdm.to_pickle("../data/pickle/tdm_sw_add.pkl")

<IPython.core.display.Javascript object>

### load

In [3]:
tdm = pd.read_pickle("../data/pickle/tdm_sw_add.pkl")
cv = pickle.load(open("../data/pickle/cv_sw_add.pkl", "rb"))
tdm

,0,1,2,3,4,5,6,7,8,9,...,229627,229628,229629,229630,229631,229632,229633,229634,229635,229636
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übrigens,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übriges,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [4]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

<IPython.core.display.Javascript object>

In [5]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
d = corpora.Dictionary()
d.id2token = id2word
# d.token2id = word2id

<IPython.core.display.Javascript object>

In [6]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    random_state=100,
    chunksize=50000,
    passes=50,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.066*"gründe" + 0.029*"afd" + 0.014*"globalisierten" + 0.012*"plakate" + 0.010*"deutsches" + 0.010*"annalena" + 0.010*"weihnachtsgans" + 0.010*"baerbock" + 0.009*"altparteien" + 0.009*"paradies"'),
 (1,
  '0.022*"klimasituation" + 0.014*"erd" + 0.012*"kleinpartei" + 0.011*"iwann" + 0.009*"meinungsfreiheit" + 0.008*"meinungsbildung" + 0.008*"wirkung" + 0.006*"italien" + 0.005*"solarkraft" + 0.005*"resultiert"'),
 (2,
  '0.061*"verzerren" + 0.018*"resultiert" + 0.015*"dankbar" + 0.012*"grüssen" + 0.011*"cdu" + 0.009*"plätze" + 0.009*"ebola" + 0.008*"letztem" + 0.008*"meinungsfreiheit" + 0.007*"finanzierung"'),
 (3,
  '0.042*"deutsches" + 0.040*"zerstörte" + 0.029*"gründe" + 0.027*"grösten" + 0.027*"lachten" + 0.027*"joseph" + 0.024*"weiterhelfen" + 0.023*"wählern" + 0.021*"mall" + 0.020*"zeichen"'),
 (4,
  '0.054*"afd" + 0.047*"paradies" + 0.046*"cdu" + 0.038*"wucht" + 0.023*"gründe" + 0.022*"spaltung" + 0.022*"wunderland" + 0.017*"paragraphen" + 0.012*"lifestyle" + 0.010*"gröst

<IPython.core.display.Javascript object>

In [ ]:
'''lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20,
    random_state=100,
    chunksize=30000,
    passes=20,
    iterations=100,
)
lda.print_topics()'''
'''[(0,
  '0.028*"afd" + 0.017*"klimaskeptikern" + 0.013*"erdachse" + 0.012*"kleinreden" + 0.011*"iwelche" + 0.010*"meinungsmacher" + 0.009*"globalisierung" + 0.008*"solarparks" + 0.007*"italienischen" + 0.007*"corona"'),
 (1,
  '0.118*"gründen" + 0.096*"deutsches" + 0.053*"zerstörungen" + 0.044*"grübeln" + 0.039*"lad" + 0.034*"journal" + 0.026*"weiterlaufen" + 0.025*"wählerstimmen" + 0.023*"geistern" + 0.023*"teilweise"'),
 (2,
  '0.013*"mehrfacher" + 0.008*"quacksalber" + 0.007*"realsatire" + 0.006*"fall" + 0.005*"argumente" + 0.005*"ehemals" + 0.005*"fox" + 0.005*"verübt" + 0.004*"schritt" + 0.004*"easy"'),
 (3,
  '0.089*"wahlparty" + 0.076*"nationalsozialistische" + 0.057*"franziskus" + 0.037*"kiddy" + 0.035*"aussage" + 0.031*"hygiene" + 0.029*"bürger" + 0.029*"mundgerecht" + 0.028*"rausgeschnitten" + 0.028*"menschenverachtend"'),
 (4,
  '0.015*"kampfdrohnen" + 0.013*"naturgesetze" + 0.010*"wall" + 0.010*"bitte" + 0.009*"dullies" + 0.007*"ahnung" + 0.007*"wärme" + 0.006*"mehren" + 0.006*"ach" + 0.006*"halbwahrheiten"'),
 (5,
  '0.011*"deutsches" + 0.011*"weiten" + 0.010*"meinungsmacher" + 0.010*"gelang" + 0.009*"lausitz" + 0.006*"iwelche" + 0.005*"prince" + 0.005*"umkreis" + 0.005*"kleinreden" + 0.005*"auto"'),
 (6,
  '0.036*"beleidigungen" + 0.031*"hervorruft" + 0.031*"repräsentiert" + 0.031*"axel" + 0.029*"vielmals" + 0.027*"cordes" + 0.027*"aufgrund" + 0.027*"neoliberalen" + 0.026*"wen" + 0.025*"aufhören"'),
 (7,
  '0.050*"mehrfacher" + 0.040*"dannach" + 0.036*"absolut" + 0.033*"levermann" + 0.032*"besonders" + 0.029*"langsame" + 0.028*"frauke" + 0.020*"ur" + 0.020*"beleidigen" + 0.020*"wetter"'),
 (8,
  '0.016*"deutsches" + 0.008*"cdu" + 0.008*"unüberlegt" + 0.008*"reformiert" + 0.007*"kreis" + 0.007*"deutsch" + 0.006*"heucheln" + 0.005*"demokrat" + 0.005*"lad" + 0.005*"bildung"'),
 (9,
  '0.071*"cdu" + 0.061*"wulfinghoff" + 0.049*"paradoxon" + 0.035*"gründen" + 0.032*"afd" + 0.029*"parallelen" + 0.025*"spamattacke" + 0.015*"light" + 0.010*"gewonnen" + 0.010*"grübeln"'),
 (10,
  '0.043*"afd" + 0.023*"rebell" + 0.022*"parteiprogrammen" + 0.019*"gittern" + 0.019*"paradoxon" + 0.017*"wunder" + 0.016*"nervig" + 0.016*"angst" + 0.015*"strommix" + 0.014*"geringste"'),
 (11,
  '0.042*"zumindest" + 0.039*"zustimmt" + 0.038*"identifizieren" + 0.037*"klatschen" + 0.037*"lämmer" + 0.034*"gesenkt" + 0.034*"heuchlerischen" + 0.033*"bezeichnen" + 0.033*"eigenem" + 0.033*"speicherbar"'),
 (12,
  '0.112*"wunderschöne" + 0.110*"afd" + 0.021*"fail" + 0.021*"gitarre" + 0.018*"komiker" + 0.017*"nebenher" + 0.016*"einsteht" + 0.015*"account" + 0.014*"übertreib" + 0.011*"antwort"'),
 (13,
  '0.134*"verzerrte" + 0.029*"dankbar" + 0.023*"rethorik" + 0.014*"geschäftsmodel" + 0.013*"bitte" + 0.011*"ebola" + 0.010*"wörter" + 0.008*"stützen" + 0.008*"guess" + 0.008*"arbeit"'),
 (14,
  '0.072*"gründen" + 0.032*"plakativ" + 0.030*"weile" + 0.025*"armer" + 0.025*"annalena" + 0.024*"halbwahrheiten" + 0.024*"baerbock" + 0.022*"gehtwaehlen" + 0.021*"schlussendlich" + 0.021*"regierungskoalition"'),
 (15,
  '0.033*"verzerrte" + 0.017*"plötzlich" + 0.016*"grütze" + 0.015*"meinungsmacher" + 0.014*"rethorik" + 0.012*"finanzindustrie" + 0.011*"letztendlich" + 0.010*"wind" + 0.009*"plötzlichen" + 0.009*"dankbar"'),
 (16,
  '0.023*"rethorik" + 0.016*"wirkungslos" + 0.014*"fakedislikes" + 0.014*"klimaskeptikern" + 0.009*"wirkung" + 0.009*"schafe" + 0.009*"profitiert" + 0.008*"tränen" + 0.008*"bezahlt" + 0.007*"schulschwänzen"'),
 (17,
  '0.051*"manipulierend" + 0.026*"klatscht" + 0.018*"lösungsvorschläge" + 0.015*"armer" + 0.014*"geschmolzen" + 0.011*"freue" + 0.011*"ur" + 0.008*"bitte" + 0.008*"cannabis" + 0.008*"tschechien"'),
 (18,
  '0.093*"löschte" + 0.036*"meinungsmacher" + 0.034*"fantastische" + 0.034*"beleidigen" + 0.033*"vergewaltigt" + 0.029*"unternehmens" + 0.028*"nocht" + 0.026*"verbleibenden" + 0.026*"stärkeren" + 0.024*"sozialdemokratische"'),
 (19,
  '0.035*"linksruck" + 0.034*"heucheln" + 0.032*"fantastisches" + 0.030*"ur" + 0.028*"zufolge" + 0.028*"beleidigt" + 0.027*"destruktiv" + 0.024*"löschte" + 0.022*"achtung" + 0.019*"irgendjemand"')]'''

### Visualize

In [ ]:
#deprecated
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim_models.prepare(topic_model=lda, corpus=corpus, dictionary=id2word)
###vis = pyLDAvis.gensim_models.prepare(topic_model=lda, corpus=corpus, dictionary=d)
###pyLDAvis.display(vis)
#vis

## 2. Topic Modeling on aggregated corpus
(bad results for single comment documents)

### skip to load, if already stored

In [11]:
dtm = pd.read_pickle("../data/pickle/split_dtm.pkl")
cv = pickle.load(open("../data/pickle/cv_split.pkl", "rb"))
dtm

,aa,aachen,aaliyah,ab,aba,abarbeiten,abartig,abartige,abartigen,abba,...,übliche,üblichen,üblicherweise,übrig,übrigen,übrigends,übrigens,übriges,übt,übung
2019-05,15,18,0,3421,11,7,35,4,4,3,...,43,63,7,260,276,15,1338,11,9,5
2019-06,8,12,0,366,1,2,4,1,0,12,...,4,19,1,25,70,1,221,1,4,1
2019-07,1,3,0,79,0,2,0,1,0,3,...,7,6,0,7,11,0,37,0,0,2
2019-08,0,3,0,65,1,0,3,0,0,3,...,4,5,0,4,11,0,30,0,0,1
2019-09,0,0,0,47,0,0,0,0,0,1,...,1,1,0,9,4,0,16,1,0,0
2019-10,0,1,0,16,1,0,0,0,0,0,...,0,0,0,3,2,0,9,0,0,0
2019-11,0,0,0,42,0,0,2,0,0,0,...,0,1,0,6,2,0,14,0,0,0
2019-12,0,0,0,33,0,0,0,1,0,0,...,2,0,0,3,6,2,13,0,0,0
2020-01,0,0,0,18,0,0,3,0,0,0,...,0,1,0,2,0,0,7,0,1,0
2020-02,0,0,0,25,0,0,0,0,0,0,...,0,0,0,2,3,0,19,0,0,0


<IPython.core.display.Javascript object>

In [12]:
tdm = dtm.transpose()
tdm.head()

,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,...,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08
aa,15,8,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,18,12,3,3,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,7,0,0,0,0
ab,3421,366,79,65,47,16,42,33,18,25,...,34,19,35,34,33,33,36,35,32,14
aba,11,1,0,1,0,1,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0


<IPython.core.display.Javascript object>

In [13]:
print(len(cv.vocabulary_))

22387


<IPython.core.display.Javascript object>

In [14]:
rows_to_drop = []
for word in list(sw):
    if word in tdm.index:
        rows_to_drop.append(word)
        cv.vocabulary_.pop(word)

print(len(rows_to_drop))

59


<IPython.core.display.Javascript object>

In [15]:
print(len(cv.vocabulary_))

22328


<IPython.core.display.Javascript object>

In [16]:
tdm.drop(index=rows_to_drop, inplace=True)

<IPython.core.display.Javascript object>

In [17]:
tdm

,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,...,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08
aa,15,8,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,18,12,3,3,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,7,0,0,0,0
ab,3421,366,79,65,47,16,42,33,18,25,...,34,19,35,34,33,33,36,35,32,14
aba,11,1,0,1,0,1,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,15,1,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
übrigens,1338,221,37,30,16,9,14,13,7,19,...,11,8,5,12,14,51,24,29,376,5
übriges,11,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
übt,9,4,0,0,0,0,0,0,1,0,...,2,0,1,2,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [19]:
pickle.dump(cv, open("../data/pickle/cv_month_sw_add.pkl", "wb"))
tdm.to_pickle("../data/pickle/tdm_month_sw_add.pkl")

<IPython.core.display.Javascript object>

### load

In [20]:
tdm = pd.read_pickle("../data/pickle/tdm_month_sw_add.pkl")
cv = pickle.load(open("../data/pickle/cv_month_sw_add.pkl", "rb"))
tdm

,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,...,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08
aa,15,8,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,18,12,3,3,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,7,0,0,0,0
ab,3421,366,79,65,47,16,42,33,18,25,...,34,19,35,34,33,33,36,35,32,14
aba,11,1,0,1,0,1,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,15,1,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
übrigens,1338,221,37,30,16,9,14,13,7,19,...,11,8,5,12,14,51,24,29,376,5
übriges,11,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
übt,9,4,0,0,0,0,0,0,1,0,...,2,0,1,2,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [21]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

<IPython.core.display.Javascript object>

In [22]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

<IPython.core.display.Javascript object>

In [23]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    passes=10,
    random_state=100,
    chunksize=30000,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.025*"grün" + 0.018*"uploaden" + 0.015*"löschen" + 0.013*"irgendeinen" + 0.013*"deutsches" + 0.013*"fantastisch" + 0.013*"linksradikale" + 0.012*"leugnung" + 0.012*"zucker" + 0.012*"fantastische"'),
 (1,
  '0.030*"afd" + 0.010*"papst" + 0.009*"wahllos" + 0.008*"globalisierten" + 0.007*"nationalsozialist" + 0.007*"corona" + 0.007*"malen" + 0.007*"altparteien" + 0.006*"deutsches" + 0.006*"klimasekte"'),
 (2,
  '0.025*"verzeihen" + 0.014*"afd" + 0.011*"malen" + 0.011*"läden" + 0.011*"löschen" + 0.010*"resultieren" + 0.009*"zerstörerische" + 0.008*"grün" + 0.008*"wunderbare" + 0.008*"deutsches"'),
 (3,
  '0.004*"resultieren" + 0.003*"cdu" + 0.003*"grün" + 0.002*"grüsse" + 0.002*"meinungsblogger" + 0.002*"dankbar" + 0.002*"wozu" + 0.002*"afd" + 0.002*"deutsches" + 0.002*"papst"'),
 (4,
  '0.015*"afd" + 0.008*"meinungsblogger" + 0.004*"manipulatoren" + 0.004*"grüsse" + 0.004*"argumente" + 0.003*"mehrere" + 0.003*"ick" + 0.003*"klimasekte" + 0.003*"wärt" + 0.003*"waldflächen"'),
 (5,

<IPython.core.display.Javascript object>

In [27]:
# TODO
lda.get_topic_terms(topicid=0, topn=10)
# lda..get_Elogbeta()

[(8485, 0.024795441),
 (19069, 0.017889254),
 (12158, 0.015223639),
 (9916, 0.013396972),
 (4214, 0.012984668),
 (6293, 0.012696419),
 (11973, 0.012596971),
 (11865, 0.012406889),
 (21748, 0.011845144),
 (6294, 0.011618693)]

<IPython.core.display.Javascript object>

In [38]:
sum([x[1] for x in lda.show_topic(topicid=0, topn=10)])

0.14545409101992846

<IPython.core.display.Javascript object>

In [51]:
sum([x[1] for x in lda.show_topic(topicid=0, topn=200)])

0.6616803085198626

<IPython.core.display.Javascript object>

In [67]:
[x for x in lda.show_topic(topicid=0, topn=50) if x[1] > 0.005]

[('grün', 0.024795441),
 ('uploaden', 0.017889254),
 ('löschen', 0.015223639),
 ('irgendeinen', 0.013396972),
 ('deutsches', 0.012984668),
 ('fantastisch', 0.012696419),
 ('linksradikale', 0.012596971),
 ('leugnung', 0.012406889),
 ('zucker', 0.011845144),
 ('fantastische', 0.011618693),
 ('hetzu', 0.01043836),
 ('achtung', 0.009971735),
 ('unternehmens', 0.009038787),
 ('beleidigen', 0.008990007),
 ('gesicht', 0.008016299),
 ('afd', 0.007864004),
 ('baerbock', 0.007821637),
 ('annalena', 0.0077502294),
 ('weigerst', 0.007322278),
 ('cordes', 0.0072866003),
 ('destruktiv', 0.0071954457),
 ('axel', 0.0070923446),
 ('startseite', 0.0069908635),
 ('plagen', 0.0069400272),
 ('verbindung', 0.006927093),
 ('rückblickend', 0.006657988),
 ('spendenskandal', 0.006465484),
 ('antidemokratischen', 0.006374246),
 ('mutiert', 0.006372146),
 ('rumsitzen', 0.0063692266),
 ('gewonnen', 0.006358788),
 ('antidemokratisch', 0.006341313),
 ('zerstörerische', 0.0061416924),
 ('mehrere', 0.006012667),
 ('mu

<IPython.core.display.Javascript object>

In [68]:
sum([x[1] for x in lda.show_topic(topicid=0, topn=50) if x[1] > 0.005])

0.41099167754873633

<IPython.core.display.Javascript object>

### save and load lda model

In [82]:
import joblib

<IPython.core.display.Javascript object>

In [84]:
joblib.dump(lda, "lda/lda_test.jl")

['lda/lda_test.jl']

<IPython.core.display.Javascript object>

In [85]:
lda_test = joblib.load("lda/lda_test.jl")

<IPython.core.display.Javascript object>

In [87]:
sum([x[1] for x in lda_test.show_topic(topicid=0, topn=200)])

0.6616803085198626

<IPython.core.display.Javascript object>

In [16]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    passes=10,
)
lda.print_topics()

[(0,
  '0.008*"resultierenden" + 0.006*"cdu" + 0.005*"meinungsdiktatur" + 0.005*"gründe" + 0.005*"grüssen" + 0.005*"deutsches" + 0.004*"afd" + 0.003*"letzte" + 0.003*"mehreren" + 0.003*"plänen"'),
 (1,
  '0.011*"cdu" + 0.008*"wtf" + 0.007*"resultierenden" + 0.006*"meinungsdiktatur" + 0.006*"grüssen" + 0.006*"paradebeispiel" + 0.006*"dankbar" + 0.005*"deutsches" + 0.005*"plänen" + 0.005*"gründe"'),
 (2,
  '0.005*"afd" + 0.005*"deutsches" + 0.005*"cdu" + 0.004*"wtf" + 0.004*"meinungsdiktatur" + 0.004*"paradebeispiel" + 0.004*"grüssen" + 0.003*"resultierenden" + 0.003*"plänen" + 0.003*"gründe"'),
 (3,
  '0.005*"cdu" + 0.004*"afd" + 0.003*"meinungsdiktatur" + 0.003*"resultierenden" + 0.003*"gründe" + 0.002*"wtf" + 0.002*"deutsches" + 0.002*"grüssen" + 0.002*"letzte" + 0.002*"klimasensitivität"'),
 (4,
  '0.008*"afd" + 0.008*"cdu" + 0.007*"resultierenden" + 0.005*"meinungsdiktatur" + 0.005*"deutsches" + 0.004*"mehreren" + 0.004*"gründe" + 0.004*"grüssen" + 0.003*"paradebeispiel" + 0.003*"pl

<IPython.core.display.Javascript object>

## 3. Topic Modeling on filtered comments

### condition:  >15words AND 10 unique words

### skip to load, if already stored

In [77]:
corpus = pd.read_pickle("../data/pickle/corpus_clean.pkl")
corpus

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
1,achtung faschisten irma lo beleidigt user ganz...
2,zeigt jahre jahre
3,doppelmoralist rezo eigenes video dezember pri...
4,richtig
...,...
229632,j pepe leg fakten falsche informationen rausha...
229633,zukunft neonazi
229634,bundestag stehen abgeordnete mandat halten unt...
229635,liebe user lasst daran hindern frei meinung ek...


<IPython.core.display.Javascript object>

In [ ]:
MIN_WORDS = 15
MIN_UNIQUE_WORDS = 10
doc_to_drop = []
progress = 0
print("")

for i in list(corpus.index):
    progress += 1
    if len(corpus.loc[i].comment.split()) < MIN_WORDS:
        doc_to_drop.append(i)
    elif len(set(corpus.loc[i].comment.split())) < MIN_UNIQUE_WORDS:
        doc_to_drop.append(i)
    if progress % 1000 == 0:
        print(f"progress: {progress}/{corpus.index.size}",end="\r")

print("")

In [100]:
corpus_filtered = corpus.drop(index=doc_to_drop)
corpus_filtered

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
3,doppelmoralist rezo eigenes video dezember pri...
6,ganzes zimmer besteht hälfte kunststoff wurde ...
10,linker terror geistige brandstifter grünen seb...
14,aktuell soviel ach sicheren sauberen atomkraft...
...,...
229619,gebe recht grünen option klimawandel stoppen g...
229621,titel sagt ja cdu argumentiert meinung aspekte...
229626,men world stammen etablierten billionen mittel...
229634,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [102]:
d = {}
for i, v in enumerate(list(corpus_filtered.index)):
    d[v] = i
corpus_filtered_clean = corpus_filtered.rename(index=d)
corpus_filtered_clean

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
1,doppelmoralist rezo eigenes video dezember pri...
2,ganzes zimmer besteht hälfte kunststoff wurde ...
3,linker terror geistige brandstifter grünen seb...
4,aktuell soviel ach sicheren sauberen atomkraft...
...,...
81889,gebe recht grünen option klimawandel stoppen g...
81890,titel sagt ja cdu argumentiert meinung aspekte...
81891,men world stammen etablierten billionen mittel...
81892,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [104]:
cv = CountVectorizer(stop_words=sw, dtype=np.int8)
filtered_cv = cv.fit_transform(corpus_filtered_clean.comment)
filtered_dtm = pd.DataFrame(filtered_cv.toarray(), columns=cv.get_feature_names())
filtered_dtm.index = corpus_filtered_clean.index
filtered_dtm

,aa,aachen,aaliyah,ab,aba,abarbeiten,abartig,abartige,abartigen,abba,...,übliche,üblichen,üblicherweise,übrig,übrigen,übrigends,übrigens,übriges,übt,übung
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81889,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [105]:
filtered_tdm = filtered_dtm.transpose()
filtered_tdm

,0,1,2,3,4,5,6,7,8,9,...,81884,81885,81886,81887,81888,81889,81890,81891,81892,81893
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übrigens,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übriges,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [106]:
pickle.dump(cv, open("../data/pickle/filtered_cv.pkl", "wb"))
filtered_tdm.to_pickle("../data/pickle/filtered_tdm.pkl")

<IPython.core.display.Javascript object>

### load

In [107]:
tdm = pd.read_pickle("../data/pickle/filtered_tdm.pkl")
cv = pickle.load(open("../data/pickle/filtered_cv.pkl", "rb"))
tdm

,0,1,2,3,4,5,6,7,8,9,...,81884,81885,81886,81887,81888,81889,81890,81891,81892,81893
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übrigens,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übriges,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
übt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [108]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

<IPython.core.display.Javascript object>

In [109]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

<IPython.core.display.Javascript object>

In [110]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    random_state=100,
    chunksize=1000,
    passes=10,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.063*"grünen" + 0.027*"platz" + 0.025*"baerbock" + 0.024*"nazi" + 0.024*"annalena" + 0.022*"hält" + 0.018*"fordern" + 0.015*"offiziell" + 0.015*"antidemokratisch" + 0.015*"gewählte"'),
 (1,
  '0.057*"zwei" + 0.047*"kleinen" + 0.045*"zusammen" + 0.027*"idioten" + 0.026*"rezo" + 0.025*"bezeichnen" + 0.025*"heutzutage" + 0.025*"läuft" + 0.025*"gesetzt" + 0.021*"eigenes"'),
 (2,
  '0.017*"klimawandel" + 0.016*"erde" + 0.015*"klima" + 0.014*"jahren" + 0.009*"jahre" + 0.009*"china" + 0.008*"heute" + 0.008*"strom" + 0.008*"auto" + 0.008*"mensch"'),
 (3,
  '0.014*"internet" + 0.011*"umweltschutz" + 0.011*"glaubst" + 0.010*"flüchtlinge" + 0.009*"euro" + 0.009*"satz" + 0.008*"seiten" + 0.007*"wegen" + 0.007*"kriege" + 0.006*"steckt"'),
 (4,
  '0.022*"lügen" + 0.019*"beleidigungen" + 0.018*"rest" + 0.017*"aufgrund" + 0.015*"werbung" + 0.015*"unterstellungen" + 0.013*"kleiner" + 0.012*"deswegen" + 0.012*"heute" + 0.011*"aufhören"'),
 (5,
  '0.042*"grünen" + 0.020*"deutschland" + 0.015*"ju

<IPython.core.display.Javascript object>

In [111]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=7,
    random_state=100,
    chunksize=10000,
    passes=50,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.064*"grünen" + 0.040*"zitat" + 0.024*"jugend" + 0.024*"grüne" + 0.024*"land" + 0.024*"weltkrieg" + 0.023*"deutschland" + 0.022*"wünschte" + 0.021*"zerbombt" + 0.020*"miene"'),
 (1,
  '0.037*"zusammen" + 0.036*"rezo" + 0.036*"video" + 0.035*"kleinen" + 0.035*"zwei" + 0.031*"gesetzt" + 0.031*"idioten" + 0.030*"heutzutage" + 0.030*"bezeichnen" + 0.030*"eigenes"'),
 (2,
  '0.010*"deutschland" + 0.008*"klimawandel" + 0.007*"klima" + 0.007*"erde" + 0.007*"jahren" + 0.007*"menschen" + 0.006*"welt" + 0.005*"geld" + 0.004*"jahre" + 0.004*"heute"'),
 (3,
  '0.010*"rezo" + 0.008*"meinung" + 0.006*"quellen" + 0.006*"fakten" + 0.006*"video" + 0.005*"bitte" + 0.004*"wissenschaftler" + 0.004*"glauben" + 0.003*"kannst" + 0.003*"argumente"'),
 (4,
  '0.031*"lügen" + 0.023*"user" + 0.022*"maniac" + 0.020*"beleidigen" + 0.019*"axel" + 0.019*"liebe" + 0.018*"beleidigungen" + 0.018*"cordes" + 0.018*"faschisten" + 0.018*"stefan"'),
 (5,
  '0.047*"video" + 0.017*"rezo" + 0.014*"cdu" + 0.011*"gut" +

<IPython.core.display.Javascript object>

In [114]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=6,
    random_state=100,
    chunksize=10000,
    passes=100,
    iterations=200,
)
lda.print_topics()

[(0,
  '0.031*"grünen" + 0.022*"lügen" + 0.022*"beleidigen" + 0.022*"zitat" + 0.017*"maniac" + 0.017*"user" + 0.013*"faschisten" + 0.013*"weltkrieg" + 0.013*"grüne" + 0.013*"deutschland"'),
 (1,
  '0.037*"rezo" + 0.036*"video" + 0.034*"kleinen" + 0.033*"läuft" + 0.033*"zwei" + 0.032*"zusammen" + 0.028*"gesetzt" + 0.028*"idioten" + 0.028*"heutzutage" + 0.027*"eigenes"'),
 (2,
  '0.013*"deutschland" + 0.008*"menschen" + 0.006*"jahren" + 0.006*"welt" + 0.006*"klimawandel" + 0.006*"klima" + 0.006*"geld" + 0.005*"erde" + 0.004*"leben" + 0.004*"heute"'),
 (3,
  '0.016*"afd" + 0.005*"fakten" + 0.004*"menschen" + 0.004*"wählt" + 0.004*"quellen" + 0.004*"meinung" + 0.003*"wissenschaftler" + 0.003*"bitte" + 0.003*"nazi" + 0.003*"glauben"'),
 (4,
  '0.013*"rest" + 0.012*"lügen" + 0.012*"aufgrund" + 0.010*"beleidigungen" + 0.010*"aufhören" + 0.009*"volksverpetzer" + 0.009*"netz" + 0.008*"methoden" + 0.008*"hetze" + 0.008*"unterstellungen"'),
 (5,
  '0.025*"video" + 0.016*"cdu" + 0.012*"rezo" + 0.0

<IPython.core.display.Javascript object>

In [116]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20,
    random_state=25,
    chunksize=1000,
    passes=10,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.050*"besonders" + 0.045*"absolut" + 0.044*"frei" + 0.040*"meinung" + 0.037*"liebe" + 0.035*"daran" + 0.030*"lasst" + 0.025*"beleidigen" + 0.024*"widerlich" + 0.024*"user"'),
 (1,
  '0.059*"video" + 0.030*"rezo" + 0.019*"menschen" + 0.017*"danke" + 0.017*"leute" + 0.015*"generation" + 0.013*"politik" + 0.013*"echt" + 0.012*"hoffe" + 0.011*"wirklich"'),
 (2,
  '0.049*"afd" + 0.021*"wählt" + 0.012*"anteil" + 0.011*"prozent" + 0.010*"atmosphäre" + 0.009*"sonne" + 0.008*"wissenschaft" + 0.008*"luft" + 0.008*"wissenschaftlich" + 0.008*"wissenschaftliche"'),
 (3,
  '0.027*"geld" + 0.022*"afd" + 0.021*"deutschland" + 0.018*"strom" + 0.014*"steuern" + 0.012*"energien" + 0.012*"energie" + 0.010*"kohle" + 0.010*"temperatur" + 0.010*"eu"'),
 (4,
  '0.027*"je" + 0.019*"tatsache" + 0.018*"vermutlich" + 0.018*"rede" + 0.015*"kind" + 0.014*"inzwischen" + 0.013*"unten" + 0.011*"tatsachen" + 0.011*"desto" + 0.011*"spielt"'),
 (5,
  '0.023*"menschen" + 0.021*"klimawandel" + 0.018*"jahren" + 0.0

<IPython.core.display.Javascript object>

In [117]:
lda.print_topics(num_words=15)

[(0,
  '0.050*"besonders" + 0.045*"absolut" + 0.044*"frei" + 0.040*"meinung" + 0.037*"liebe" + 0.035*"daran" + 0.030*"lasst" + 0.025*"beleidigen" + 0.024*"widerlich" + 0.024*"user" + 0.022*"verkommen" + 0.022*"andersdenkenden" + 0.022*"hindern" + 0.021*"ekelhaften" + 0.020*"steigt"'),
 (1,
  '0.059*"video" + 0.030*"rezo" + 0.019*"menschen" + 0.017*"danke" + 0.017*"leute" + 0.015*"generation" + 0.013*"politik" + 0.013*"echt" + 0.012*"hoffe" + 0.011*"wirklich" + 0.011*"junge" + 0.010*"endlich" + 0.010*"zukunft" + 0.009*"gut" + 0.009*"lass"'),
 (2,
  '0.049*"afd" + 0.021*"wählt" + 0.012*"anteil" + 0.011*"prozent" + 0.010*"atmosphäre" + 0.009*"sonne" + 0.008*"wissenschaft" + 0.008*"luft" + 0.008*"wissenschaftlich" + 0.008*"wissenschaftliche" + 0.007*"co²" + 0.007*"halte" + 0.007*"ipcc" + 0.007*"klimawandel" + 0.007*"schwachsinn"'),
 (3,
  '0.027*"geld" + 0.022*"afd" + 0.021*"deutschland" + 0.018*"strom" + 0.014*"steuern" + 0.012*"energien" + 0.012*"energie" + 0.010*"kohle" + 0.010*"tempera

<IPython.core.display.Javascript object>

# iterations/testing params
## 1.
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    random_state=100,
    chunksize=1000,
    passes=10,
    iterations=100,
)
\
\
(0,
  '0.063*"grünen" + 0.027*"platz" + 0.025*"baerbock" + 0.024*"nazi" + 0.024*"annalena" + 0.022*"hält" + 0.018*"fordern" + 0.015*"offiziell" + 0.015*"antidemokratisch" + 0.015*"gewählte"'),
 \
 (1,
  '0.057*"zwei" + 0.047*"kleinen" + 0.045*"zusammen" + 0.027*"idioten" + 0.026*"rezo" + 0.025*"bezeichnen" + 0.025*"heutzutage" + 0.025*"läuft" + 0.025*"gesetzt" + 0.021*"eigenes"'),
 \
 (2,
  '0.017*"klimawandel" + 0.016*"erde" + 0.015*"klima" + 0.014*"jahren" + 0.009*"jahre" + 0.009*"china" + 0.008*"heute" + 0.008*"strom" + 0.008*"auto" + 0.008*"mensch"'),
 \
 (3,
  '0.014*"internet" + 0.011*"umweltschutz" + 0.011*"glaubst" + 0.010*"flüchtlinge" + 0.009*"euro" + 0.009*"satz" + 0.008*"seiten" + 0.007*"wegen" + 0.007*"kriege" + 0.006*"steckt"'),
 \
 (4,
  '0.022*"lügen" + 0.019*"beleidigungen" + 0.018*"rest" + 0.017*"aufgrund" + 0.015*"werbung" + 0.015*"unterstellungen" + 0.013*"kleiner" + 0.012*"deswegen" + 0.012*"heute" + 0.011*"aufhören"'),
 \
 (5,
  '0.042*"grünen" + 0.020*"deutschland" + 0.015*"jugend" + 0.014*"rezo" + 0.013*"land" + 0.013*"rezos" + 0.012*"reich" + 0.012*"grüne" + 0.012*"zitat" + 0.008*"richtig"'),
 \
 (6,
  '0.080*"afd" + 0.042*"partei" + 0.038*"grünen" + 0.016*"linke" + 0.014*"spd" + 0.012*"linken" + 0.012*"wählt" + 0.009*"fdp" + 0.009*"parteien" + 0.009*"grüne"'),
 \
 (7,
  '0.019*"liebe" + 0.019*"user" + 0.018*"besonders" + 0.017*"meinung" + 0.016*"beleidigen" + 0.014*"frei" + 0.013*"absolut" + 0.013*"daran" + 0.012*"beleidigt" + 0.011*"lasst"'),
 \
 (8,
  '0.018*"video" + 0.012*"cdu" + 0.011*"rezo" + 0.009*"menschen" + 0.009*"gut" + 0.008*"meinung" + 0.007*"leute" + 0.007*"politik" + 0.007*"wählen" + 0.006*"wirklich"'),
 \
 (9,
  '0.016*"deutschland" + 0.008*"geld" + 0.007*"menschen" + 0.006*"zb" + 0.005*"etc" + 0.005*"beispiel" + 0.005*"klimawandel" + 0.004*"usa" + 0.004*"welt" + 0.004*"weniger"')
## 2.
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=7,
    random_state=100,
    chunksize=10000,
    passes=50,
    iterations=100,
)
 \
  \
(0,
  '0.064*"grünen" + 0.040*"zitat" + 0.024*"jugend" + 0.024*"grüne" + 0.024*"land" + 0.024*"weltkrieg" + 0.023*"deutschland" + 0.022*"wünschte" + 0.021*"zerbombt" + 0.020*"miene"'),
  \
  (1,
  '0.037*"zusammen" + 0.036*"rezo" + 0.036*"video" + 0.035*"kleinen" + 0.035*"zwei" + 0.031*"gesetzt" + 0.031*"idioten" + 0.030*"heutzutage" + 0.030*"bezeichnen" + 0.030*"eigenes"'),
  \
  (2,
  '0.010*"deutschland" + 0.008*"klimawandel" + 0.007*"klima" + 0.007*"erde" + 0.007*"jahren" + 0.007*"menschen" + 0.006*"welt" + 0.005*"geld" + 0.004*"jahre" + 0.004*"heute"'),
 \
 (3,
  '0.010*"rezo" + 0.008*"meinung" + 0.006*"quellen" + 0.006*"fakten" + 0.006*"video" + 0.005*"bitte" + 0.004*"wissenschaftler" + 0.004*"glauben" + 0.003*"kannst" + 0.003*"argumente"'),
 \
 (4,
  '0.031*"lügen" + 0.023*"user" + 0.022*"maniac" + 0.020*"beleidigen" + 0.019*"axel" + 0.019*"liebe" + 0.018*"beleidigungen" + 0.018*"cordes" + 0.018*"faschisten" + 0.018*"stefan"'),
 \
 (5,
  '0.047*"video" + 0.017*"rezo" + 0.014*"cdu" + 0.011*"gut" + 0.010*"danke" + 0.010*"leute" + 0.010*"politik" + 0.009*"wählen" + 0.009*"menschen" + 0.008*"finde"'),
 \
 (6,
  '0.015*"afd" + 0.011*"cdu" + 0.011*"partei" + 0.010*"grünen" + 0.007*"menschen" + 0.006*"deutschland" + 0.006*"parteien" + 0.006*"politik" + 0.005*"wählen" + 0.005*"spd"')
 ## 3.
 lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=6,
    random_state=100,
    chunksize=10000,
    passes=100,
    iterations=200,
)
 \
 \
 (0,
  '0.031*"grünen" + 0.022*"lügen" + 0.022*"beleidigen" + 0.022*"zitat" + 0.017*"maniac" + 0.017*"user" + 0.013*"faschisten" + 0.013*"weltkrieg" + 0.013*"grüne" + 0.013*"deutschland"'),
  \
  (1,
  '0.037*"rezo" + 0.036*"video" + 0.034*"kleinen" + 0.033*"läuft" + 0.033*"zwei" + 0.032*"zusammen" + 0.028*"gesetzt" + 0.028*"idioten" + 0.028*"heutzutage" + 0.027*"eigenes"'),
  \
  (2,
  '0.013*"deutschland" + 0.008*"menschen" + 0.006*"jahren" + 0.006*"welt" + 0.006*"klimawandel" + 0.006*"klima" + 0.006*"geld" + 0.005*"erde" + 0.004*"leben" + 0.004*"heute"'),
  \
  (3,
  '0.016*"afd" + 0.005*"fakten" + 0.004*"menschen" + 0.004*"wählt" + 0.004*"quellen" + 0.004*"meinung" + 0.003*"wissenschaftler" + 0.003*"bitte" + 0.003*"nazi" + 0.003*"glauben"'),
  \
  (4,
  '0.013*"rest" + 0.012*"lügen" + 0.012*"aufgrund" + 0.010*"beleidigungen" + 0.010*"aufhören" + 0.009*"volksverpetzer" + 0.009*"netz" + 0.008*"methoden" + 0.008*"hetze" + 0.008*"unterstellungen"'),
  \
  (5,
  '0.025*"video" + 0.016*"cdu" + 0.012*"rezo" + 0.009*"wählen" + 0.008*"partei" + 0.008*"politik" + 0.008*"gut" + 0.007*"grünen" + 0.007*"leute" + 0.007*"parteien"')
## 4.
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20,
    random_state=25,
    chunksize=1000,
    passes=10,
    iterations=100,
)
 \
 \
(0,
  '0.050*"besonders" + 0.045*"absolut" + 0.044*"frei" + 0.040*"meinung" + 0.037*"liebe" + 0.035*"daran" + 0.030*"lasst" + 0.025*"beleidigen" + 0.024*"widerlich" + 0.024*"user"'),
  \
  (1,
  '0.059*"video" + 0.030*"rezo" + 0.019*"menschen" + 0.017*"danke" + 0.017*"leute" + 0.015*"generation" + 0.013*"politik" + 0.013*"echt" + 0.012*"hoffe" + 0.011*"wirklich"'),
  \
  (2,
  '0.049*"afd" + 0.021*"wählt" + 0.012*"anteil" + 0.011*"prozent" + 0.010*"atmosphäre" + 0.009*"sonne" + 0.008*"wissenschaft" + 0.008*"luft" + 0.008*"wissenschaftlich" + 0.008*"wissenschaftliche"'),
  \
  (3,
  '0.027*"geld" + 0.022*"afd" + 0.021*"deutschland" + 0.018*"strom" + 0.014*"steuern" + 0.012*"energien" + 0.012*"energie" + 0.010*"kohle" + 0.010*"temperatur" + 0.010*"eu"'),
  \
  (4,
  '0.027*"je" + 0.019*"tatsache" + 0.018*"vermutlich" + 0.018*"rede" + 0.015*"kind" + 0.014*"inzwischen" + 0.013*"unten" + 0.011*"tatsachen" + 0.011*"desto" + 0.011*"spielt"'),
  \
  (5,
  '0.023*"menschen" + 0.021*"klimawandel" + 0.018*"jahren" + 0.018*"erde" + 0.016*"klima" + 0.012*"jahre" + 0.011*"heute" + 0.010*"mensch" + 0.009*"welt" + 0.007*"gab"'),
  \
  (6,
  '0.104*"grünen" + 0.030*"platz" + 0.030*"weiterhin" + 0.027*"baerbock" + 0.026*"annalena" + 0.022*"benutzt" + 0.019*"fordern" + 0.018*"kanal" + 0.016*"offiziell" + 0.016*"antidemokratisch"'),
  \
  (7,
  '0.070*"lügen" + 0.039*"lo" + 0.034*"beleidigungen" + 0.030*"verbreiten" + 0.030*"deswegen" + 0.027*"verbreitet" + 0.027*"heute" + 0.026*"eike" + 0.025*"faschisten" + 0.024*"zukunft"'),
  \
  (8,
  '0.084*"glauben" + 0.045*"meinungen" + 0.030*"passt" + 0.028*"rechten" + 0.028*"verantwortung" + 0.022*"gern" + 0.020*"rum" + 0.019*"suchen" + 0.018*"studie" + 0.017*"mainstream"'),
 \
 (9,
  '0.045*"zahlen" + 0.024*"vollkommen" + 0.017*"lügner" + 0.017*"versteht" + 0.016*"netz" + 0.016*"beiden" + 0.014*"hetze" + 0.012*"forschung" + 0.012*"birne" + 0.011*"ressourcen"'),
  \
  (10,
  '0.085*"kleinen" + 0.067*"zwei" + 0.061*"läuft" + 0.059*"zusammen" + 0.049*"idioten" + 0.046*"bezeichnen" + 0.045*"heutzutage" + 0.044*"gesetzt" + 0.038*"eigenes" + 0.035*"spiel"'),
  \
  (11,
  '0.092*"grünen" + 0.049*"zitat" + 0.042*"grüne" + 0.036*"deutschland" + 0.036*"jugend" + 0.034*"land" + 0.027*"irgendwelche" + 0.024*"woher" + 0.019*"weltkrieg" + 0.019*"linker"'),
  \
  (12,
  '0.041*"rest" + 0.036*"aufhören" + 0.029*"aufgrund" + 0.027*"methoden" + 0.026*"stefan" + 0.025*"beleidigungen" + 0.023*"axel" + 0.022*"lena" + 0.022*"lügen" + 0.022*"vogel"'),
  \
  (13,
  '0.040*"deutschland" + 0.021*"usa" + 0.019*"welt" + 0.015*"auto" + 0.015*"china" + 0.014*"krieg" + 0.013*"menschen" + 0.012*"richtig" + 0.011*"länder" + 0.009*"rezo"'),
  \
  (14,
  '0.028*"schule" + 0.015*"schüler" + 0.013*"gebe" + 0.012*"beeinflussen" + 0.010*"einzigen" + 0.010*"behauptung" + 0.010*"demo" + 0.010*"future" + 0.009*"friday" + 0.009*"kommentaren"'),
  \
  (15,
  '0.031*"video" + 0.020*"rezo" + 0.015*"meinung" + 0.013*"quellen" + 0.012*"gut" + 0.011*"fakten" + 0.010*"wissenschaftler" + 0.009*"thema" + 0.007*"klar" + 0.007*"falsch"'),
  \
  (16,
  '0.020*"cdu" + 0.015*"partei" + 0.013*"grünen" + 0.012*"parteien" + 0.012*"wählen" + 0.010*"politik" + 0.010*"afd" + 0.008*"leute" + 0.007*"spd" + 0.007*"menschen"'),
 \
 (17,
  '0.033*"verhindern" + 0.032*"beleidigen" + 0.031*"behauptungen" + 0.030*"sachlich" + 0.025*"neutral" + 0.024*"unterstellen" + 0.023*"mehreren" + 0.023*"menschen" + 0.021*"account" + 0.020*"normalen"'),
  \
  (18,
  '0.037*"mario" + 0.027*"ströer" + 0.026*"beleidigt" + 0.025*"rezo" + 0.021*"kritisch" + 0.018*"impressum" + 0.017*"dummheit" + 0.015*"bitte" + 0.015*"leute" + 0.015*"möchtest"'),
  \
  (19,
  '0.126*"nazi" + 0.102*"aussage" + 0.085*"frau" + 0.048*"umweltschutz" + 0.042*"hält" + 0.034*"janich" + 0.034*"kinder" + 0.026*"reaktion" + 0.025*"widerlegen" + 0.025*"gelernt"')

## 4. Topic Modeling on filtered comments (Nouns and Adjectives - limited to sentiWS)

### condition: >15words AND 10 unique words (only nouns and adjectives)

### skip to load, if already stored

In [4]:
corpus = pd.read_pickle("../data/pickle/corpus_clean.pkl")
corpus

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
1,achtung faschisten irma lo beleidigt user ganz...
2,zeigt jahre jahre
3,doppelmoralist rezo eigenes video dezember pri...
4,richtig
...,...
229632,j pepe leg fakten falsche informationen rausha...
229633,zukunft neonazi
229634,bundestag stehen abgeordnete mandat halten unt...
229635,liebe user lasst daran hindern frei meinung ek...


<IPython.core.display.Javascript object>

In [5]:
MIN_WORDS = 15
MIN_UNIQUE_WORDS = 10
doc_to_drop = []
progress = 0
print("")

for i in list(corpus.index):
    progress += 1
    if len(corpus.loc[i].comment.split()) < MIN_WORDS:
        doc_to_drop.append(i)
    elif len(set(corpus.loc[i].comment.split())) < MIN_UNIQUE_WORDS:
        doc_to_drop.append(i)
    if progress % 1000 == 0:
        print(f"progress: {progress}/{corpus.index.size}",end="\r")
        
print("")


progress: 229000/229637


<IPython.core.display.Javascript object>

In [6]:
corpus_filtered = corpus.drop(index=doc_to_drop)
corpus_filtered

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
3,doppelmoralist rezo eigenes video dezember pri...
6,ganzes zimmer besteht hälfte kunststoff wurde ...
10,linker terror geistige brandstifter grünen seb...
14,aktuell soviel ach sicheren sauberen atomkraft...
...,...
229619,gebe recht grünen option klimawandel stoppen g...
229621,titel sagt ja cdu argumentiert meinung aspekte...
229626,men world stammen etablierten billionen mittel...
229634,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [7]:
d = {}
for i, v in enumerate(list(corpus_filtered.index)):
    d[v] = i
corpus_filtered_clean = corpus_filtered.rename(index=d)
corpus_filtered_clean

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
1,doppelmoralist rezo eigenes video dezember pri...
2,ganzes zimmer besteht hälfte kunststoff wurde ...
3,linker terror geistige brandstifter grünen seb...
4,aktuell soviel ach sicheren sauberen atomkraft...
...,...
81889,gebe recht grünen option klimawandel stoppen g...
81890,titel sagt ja cdu argumentiert meinung aspekte...
81891,men world stammen etablierten billionen mittel...
81892,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [8]:
nr_words = sum(
    [
        len(corpus_filtered_clean.loc[x].comment.split())
        for x in list(corpus_filtered_clean.index)
    ]
)
nr_words

2875820

<IPython.core.display.Javascript object>

In [9]:
nouns_and_adjectives = []

with open("../data/sentiWS/train.csv", "r") as f:
    f_reader = csv.reader(f)
    for line in f:
        line = line.split(",")
        if line[1] == "nn" or line[1] == "adjx":
            nouns_and_adjectives.append(line[0])
print(len(nouns_and_adjectives))
# nouns:3130
# adjectives: 23911
# nouns+adjectives:26041

26041


<IPython.core.display.Javascript object>

In [10]:
def only_nouns_and_adjectives(s):
    ret = []
    for w in s.split():
        if w in nouns_and_adjectives:
            ret.append(w)
    return " ".join(ret)

<IPython.core.display.Javascript object>

In [11]:
progress = 0
print("")

for k in list(corpus_filtered_clean.index):
    progress += 1
    corpus_filtered_clean.loc[k].comment = only_nouns_and_adjectives(
        corpus_filtered_clean.loc[k].comment
    )
    if progress % 1000 == 0:
        print(f"progress: {progress}/{len(corpus_filtered_clean.index)}",end="\r")
        
print("")
corpus_filtered


progress: 81000/81894


,comment
0,feier sowas endlich jemand alten cdu stimme ni...
3,doppelmoralist rezo eigenes video dezember pri...
6,ganzes zimmer besteht hälfte kunststoff wurde ...
10,linker terror geistige brandstifter grünen seb...
14,aktuell soviel ach sicheren sauberen atomkraft...
...,...
229619,gebe recht grünen option klimawandel stoppen g...
229621,titel sagt ja cdu argumentiert meinung aspekte...
229626,men world stammen etablierten billionen mittel...
229634,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [12]:
nr_words = sum(
    [
        len(corpus_filtered_clean.loc[x].comment.split())
        for x in list(corpus_filtered_clean.index)
    ]
)
nr_words

424093

<IPython.core.display.Javascript object>

In [13]:
cv = CountVectorizer(stop_words=sw, dtype=np.int8)
filtered_nouns_adjectives_cv = cv.fit_transform(corpus_filtered_clean.comment)
filtered_noun_adjectives_dtm = pd.DataFrame(
    filtered_nouns_adjectives_cv.toarray(), columns=cv.get_feature_names()
)
filtered_noun_adjectives_dtm.index = corpus_filtered_clean.index
filtered_noun_adjectives_dtm

,abbau,abbauen,abfall,abfälle,abfällig,abgebrochen,abgesichert,abgrund,abgründe,abhängig,...,überzeugend,überzeugende,überzeugenden,überzeugender,überzeugendes,überzeugt,überzeugte,überzeugter,überzeugung,überzeugungen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [14]:
filtered_noun_adjectives_tdm = filtered_noun_adjectives_dtm.transpose()
filtered_noun_adjectives_tdm

,0,1,2,3,4,5,6,7,8,9,...,81884,81885,81886,81887,81888,81889,81890,81891,81892,81893
abbau,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbauen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abfall,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abfälle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abfällig,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
überzeugt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
überzeugte,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
überzeugter,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
überzeugung,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [15]:
pickle.dump(cv, open("../data/pickle/filtered_nouns_adjectives_cv.pkl", "wb"))
filtered_noun_adjectives_tdm.to_pickle("../data/pickle/filtered_noun_adjectives_tdm.pkl")

<IPython.core.display.Javascript object>

### load

In [55]:
tdm = pd.read_pickle("../data/pickle/filtered_noun_adjectives_tdm.pkl")
cv = pickle.load(open("../data/pickle/filtered_nouns_adjectives_cv.pkl", "rb"))
tdm

,0,1,2,3,4,5,6,7,8,9,...,81884,81885,81886,81887,81888,81889,81890,81891,81892,81893
abbau,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbauen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abfall,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abfälle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abfällig,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
überzeugt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
überzeugte,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
überzeugter,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
überzeugung,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [56]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

<IPython.core.display.Javascript object>

In [57]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

<IPython.core.display.Javascript object>

In [58]:
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=7,
    random_state=100,
    chunksize=20000,
    passes=80,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.117*"absolut" + 0.096*"liebe" + 0.092*"lügen" + 0.083*"frei" + 0.074*"faschisten" + 0.066*"kritik" + 0.049*"einheitliches" + 0.049*"fiesen" + 0.039*"propaganda" + 0.035*"bomben"'),
 (1,
  '0.074*"gut" + 0.029*"recht" + 0.024*"klar" + 0.022*"besser" + 0.022*"wichtig" + 0.020*"richtig" + 0.020*"leider" + 0.018*"super" + 0.017*"terror" + 0.016*"glaube"'),
 (2,
  '0.149*"lügen" + 0.095*"beleidigungen" + 0.065*"respekt" + 0.043*"massiven" + 0.042*"faschistischen" + 0.041*"diffamierungen" + 0.040*"lügner" + 0.039*"leider" + 0.032*"schön" + 0.028*"faschist"'),
 (3,
  '0.143*"kleinen" + 0.115*"idioten" + 0.106*"freunden" + 0.057*"rechten" + 0.052*"glück" + 0.042*"glauben" + 0.033*"intelligent" + 0.030*"anständig" + 0.022*"unsinn" + 0.021*"intelligenz"'),
 (4,
  '0.021*"krieg" + 0.019*"schuld" + 0.014*"anteil" + 0.012*"aktuell" + 0.012*"möglich" + 0.011*"verantwortlich" + 0.011*"gut" + 0.011*"problem" + 0.011*"leider" + 0.010*"flüchtlinge"'),
 (5,
  '0.065*"richtig" + 0.046*"angst" + 

<IPython.core.display.Javascript object>

### 1. only nouns
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=7,
    random_state=100,
    chunksize=20000,
    passes=80,
    iterations=100,
)
lda.print_topics()
\
\
(0,
  '0.164*"recht" + 0.064*"respekt" + 0.032*"sorgen" + 0.032*"interessen" + 0.032*"müll" + 0.028*"krieg" + 0.027*"mühe" + 0.027*"sterben" + 0.025*"kriege" + 0.025*"aufmerksamkeit"'),
 \
 (1,
  '0.080*"schuld" + 0.075*"propaganda" + 0.053*"probleme" + 0.051*"anteil" + 0.035*"lügner" + 0.026*"wissen" + 0.025*"konsens" + 0.024*"kriegen" + 0.023*"sinne" + 0.023*"wohlstand"'),
 \
 (2,
  '0.114*"ende" + 0.055*"glück" + 0.050*"fehler" + 0.039*"ziel" + 0.037*"zerstörung" + 0.033*"inkompetenz" + 0.026*"lösungen" + 0.025*"probleme" + 0.024*"wissen" + 0.022*"gefallen"'),
 \
 (3,
  '0.146*"idioten" + 0.144*"lügen" + 0.140*"faschisten" + 0.135*"freunden" + 0.112*"beleidigungen" + 0.078*"kritik" + 0.062*"diffamierungen" + 0.057*"wissen" + 0.049*"bomben" + 0.019*"sicherheit"'),
 \
 (4,
  '0.144*"problem" + 0.092*"glaube" + 0.081*"angst" + 0.069*"terror" + 0.054*"krieg" + 0.053*"bildung" + 0.038*"sinn" + 0.035*"nutzen" + 0.029*"dreck" + 0.027*"lösung"'),
 \
 (5,
  '0.213*"lügen" + 0.173*"liebe" + 0.065*"faschist" + 0.047*"flüchtlinge" + 0.032*"hoffnung" + 0.030*"streiken" + 0.030*"beleidigungen" + 0.025*"interesse" + 0.020*"armut" + 0.020*"heuchler"'),
 \
 (6,
  '0.097*"glauben" + 0.085*"wahrheit" + 0.057*"rechten" + 0.039*"verantwortung" + 0.038*"rechts" + 0.037*"wert" + 0.033*"unsinn" + 0.032*"rechte" + 0.031*"experten" + 0.031*"wissen"')
  
### 2. only nouns and adjectives
lda = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=7,
    random_state=100,
    chunksize=20000,
    passes=80,
    iterations=100,
)
lda.print_topics()
\
\
(0,
  '0.117*"absolut" + 0.096*"liebe" + 0.092*"lügen" + 0.083*"frei" + 0.074*"faschisten" + 0.066*"kritik" + 0.049*"einheitliches" + 0.049*"fiesen" + 0.039*"propaganda" + 0.035*"bomben"'),
 \
 (1,
  '0.074*"gut" + 0.029*"recht" + 0.024*"klar" + 0.022*"besser" + 0.022*"wichtig" + 0.020*"richtig" + 0.020*"leider" + 0.018*"super" + 0.017*"terror" + 0.016*"glaube"'),
 \
 (2,
  '0.149*"lügen" + 0.095*"beleidigungen" + 0.065*"respekt" + 0.043*"massiven" + 0.042*"faschistischen" + 0.041*"diffamierungen" + 0.040*"lügner" + 0.039*"leider" + 0.032*"schön" + 0.028*"faschist"'),
 \
 (3,
  '0.143*"kleinen" + 0.115*"idioten" + 0.106*"freunden" + 0.057*"rechten" + 0.052*"glück" + 0.042*"glauben" + 0.033*"intelligent" + 0.030*"anständig" + 0.022*"unsinn" + 0.021*"intelligenz"'),
 \
 (4,
  '0.021*"krieg" + 0.019*"schuld" + 0.014*"anteil" + 0.012*"aktuell" + 0.012*"möglich" + 0.011*"verantwortlich" + 0.011*"gut" + 0.011*"problem" + 0.011*"leider" + 0.010*"flüchtlinge"'),
 \
 (5,
  '0.065*"richtig" + 0.046*"angst" + 0.044*"reich" + 0.040*"alten" + 0.029*"reichen" + 0.026*"toll" + 0.021*"traurig" + 0.021*"arm" + 0.020*"schlimm" + 0.017*"krass"'),
 \
 (6,
  '0.026*"falsch" + 0.023*"wissen" + 0.020*"leider" + 0.020*"ende" + 0.019*"besser" + 0.018*"gut" + 0.017*"dumm" + 0.016*"komplett" + 0.015*"klar" + 0.015*"problem"')

## 5. Topic Modeling on filtered comments using TF-IDF

### condition:  >15words AND 10 unique words

In [ ]:
corpus = pd.read_pickle("../data/pickle/corpus_clean.pkl")
corpus

In [6]:
MIN_WORDS = 15
MIN_UNIQUE_WORDS = 10
doc_to_drop = []
progress = 0
print("")

for i in list(corpus.index):
    progress += 1
    if len(corpus.loc[i].comment.split()) < MIN_WORDS:
        doc_to_drop.append(i)
    elif len(set(corpus.loc[i].comment.split())) < MIN_UNIQUE_WORDS:
        doc_to_drop.append(i)
    if progress % 1000 == 0:
        print(f"progress: {progress}/{corpus.index.size}",end="\r")

print("")


progress: 229000/229576


<IPython.core.display.Javascript object>

In [7]:
corpus_filtered = corpus.drop(index=doc_to_drop)
corpus_filtered

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
3,doppelmoralist rezo eigenes video dezember pri...
6,ganzes zimmer besteht hälfte kunststoff wurde ...
10,linker terror geistige brandstifter grünen seb...
14,aktuell soviel ach sicheren sauberen atomkraft...
...,...
229559,gebe recht grünen option klimawandel stoppen g...
229561,titel sagt ja cdu argumentiert meinung aspekte...
229566,men world stammen etablierten billionen mittel...
229573,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [8]:
d = {}
for i, v in enumerate(list(corpus_filtered.index)):
    d[v] = i
corpus_filtered_clean = corpus_filtered.rename(index=d)
corpus_filtered_clean

,comment
0,feier sowas endlich jemand alten cdu stimme ni...
1,doppelmoralist rezo eigenes video dezember pri...
2,ganzes zimmer besteht hälfte kunststoff wurde ...
3,linker terror geistige brandstifter grünen seb...
4,aktuell soviel ach sicheren sauberen atomkraft...
...,...
81893,gebe recht grünen option klimawandel stoppen g...
81894,titel sagt ja cdu argumentiert meinung aspekte...
81895,men world stammen etablierten billionen mittel...
81896,bundestag stehen abgeordnete mandat halten unt...


<IPython.core.display.Javascript object>

In [9]:
tfidf_vectorizer = TfidfVectorizer(stop_words=sw)
tfidf_wm = tfidf_vectorizer.fit_transform(corpus_filtered_clean.comment)
tfidf = pd.DataFrame(tfidf_wm.toarray(), columns=tfidf_vectorizer.get_feature_names())
tfidf.index = corpus_filtered_clean.index
tfidf

,aa,aachen,aaliyah,ab,aba,abarbeiten,abartig,abartige,abartigen,abba,...,übliche,üblichen,üblicherweise,übrig,übrigen,übrigends,übrigens,übriges,übt,übung
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.093165,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81893,0.0,0.0,0.0,0.155822,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81894,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81895,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81896,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [10]:
tfidf = tfidf.transpose()
tfidf

,0,1,2,3,4,5,6,7,8,9,...,81888,81889,81890,81891,81892,81893,81894,81895,81896,81897
aa,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
aachen,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
aaliyah,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
ab,0.0,0.0,0.0,0.0,0.093165,0.0,0.0,0.0,0.0,0.0,...,0.0,0.147903,0.0,0.0,0.0,0.155822,0.0,0.0,0.0,0.0
aba,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übrigends,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
übrigens,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
übriges,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
übt,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [11]:
sparse_counts = scipy.sparse.csr_matrix(tfidf)
corpus_sparse = matutils.Sparse2Corpus(sparse_counts)

<IPython.core.display.Javascript object>

In [12]:
id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())

<IPython.core.display.Javascript object>

In [14]:
lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=10,
    random_state=100,
    chunksize=10000,
    passes=50,
    iterations=100,
)
lda.print_topics()

[(0,
  '0.030*"gespielt" + 0.030*"privat" + 0.029*"maskenpflicht" + 0.029*"couch" + 0.029*"eingehalten" + 0.029*"geltenden" + 0.029*"abstandsregeln" + 0.029*"coronaregeln" + 0.029*"bezeichnen" + 0.029*"freunden"'),
 (1,
  '0.036*"zitat" + 0.022*"zerbombt" + 0.022*"miene" + 0.022*"existenzberechtigung" + 0.022*"waziri" + 0.022*"weltkrieg" + 0.022*"wünschte" + 0.021*"grünen" + 0.021*"jugend" + 0.020*"grüne"'),
 (2,
  '0.004*"klimawandel" + 0.004*"deutschland" + 0.004*"klima" + 0.004*"erde" + 0.003*"strom" + 0.003*"auto" + 0.003*"china" + 0.003*"menschen" + 0.003*"usa" + 0.003*"energie"'),
 (3,
  '0.021*"lügen" + 0.019*"faschisten" + 0.016*"volksverpetzer" + 0.014*"lo" + 0.013*"hetze" + 0.012*"cordes" + 0.012*"axel" + 0.012*"widerliche" + 0.012*"stefan" + 0.011*"heute"'),
 (4,
  '0.025*"unterstellen" + 0.025*"subjekte" + 0.024*"verhindern" + 0.024*"spalten" + 0.024*"gegeneinander" + 0.024*"meinungsaustausch" + 0.023*"normalen" + 0.023*"fiesen" + 0.023*"einheitliches" + 0.023*"erkennungsme

<IPython.core.display.Javascript object>

In [15]:
lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=20,
    random_state=100,
    passes=100,
    iterations=200,
)
lda.print_topics()

[(0,
  '0.010*"rechten" + 0.010*"unterwegs" + 0.008*"kapitalismus" + 0.008*"glauben" + 0.007*"studie" + 0.007*"programm" + 0.007*"frieden" + 0.007*"afdwähler" + 0.006*"haha" + 0.006*"nato"'),
 (1,
  '0.053*"zitat" + 0.038*"grünen" + 0.028*"volkstod" + 0.028*"geistige" + 0.028*"sebastian" + 0.028*"zuwanderung" + 0.028*"striegel" + 0.027*"linker" + 0.026*"brandstifter" + 0.026*"terror"'),
 (2,
  '0.007*"deutschland" + 0.006*"grünen" + 0.006*"cdu" + 0.006*"menschen" + 0.005*"partei" + 0.005*"video" + 0.005*"afd" + 0.005*"klimawandel" + 0.005*"jahren" + 0.004*"welt"'),
 (3,
  '0.022*"volksverpetzer" + 0.016*"andersdenkende" + 0.015*"netz" + 0.014*"fakenews" + 0.014*"hetze" + 0.013*"diffamieren" + 0.012*"meinungsfreiheit" + 0.012*"denunzieren" + 0.012*"üben" + 0.011*"widerliche"'),
 (4,
  '0.028*"gegeneinander" + 0.027*"spalten" + 0.027*"unterstellen" + 0.026*"meinungsaustausch" + 0.026*"normalen" + 0.026*"einheitliches" + 0.026*"fiesen" + 0.026*"erkennungsmerkmal" + 0.026*"aufwiegeln" + 0.

<IPython.core.display.Javascript object>

In [16]:
lda.print_topics(num_words=15)

[(0,
  '0.010*"rechten" + 0.010*"unterwegs" + 0.008*"kapitalismus" + 0.008*"glauben" + 0.007*"studie" + 0.007*"programm" + 0.007*"frieden" + 0.007*"afdwähler" + 0.006*"haha" + 0.006*"nato" + 0.006*"wahlomat" + 0.006*"populismus" + 0.006*"sobald" + 0.006*"weltbild" + 0.005*"hör"'),
 (1,
  '0.053*"zitat" + 0.038*"grünen" + 0.028*"volkstod" + 0.028*"geistige" + 0.028*"sebastian" + 0.028*"zuwanderung" + 0.028*"striegel" + 0.027*"linker" + 0.026*"brandstifter" + 0.026*"terror" + 0.026*"weltkrieg" + 0.026*"zerbombt" + 0.026*"miene" + 0.026*"existenzberechtigung" + 0.026*"waziri"'),
 (2,
  '0.007*"deutschland" + 0.006*"grünen" + 0.006*"cdu" + 0.006*"menschen" + 0.005*"partei" + 0.005*"video" + 0.005*"afd" + 0.005*"klimawandel" + 0.005*"jahren" + 0.004*"welt" + 0.004*"parteien" + 0.004*"klima" + 0.004*"gut" + 0.004*"rezo" + 0.004*"politik"'),
 (3,
  '0.022*"volksverpetzer" + 0.016*"andersdenkende" + 0.015*"netz" + 0.014*"fakenews" + 0.014*"hetze" + 0.013*"diffamieren" + 0.012*"meinungsfreiheit

<IPython.core.display.Javascript object>

In [17]:
lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=7,
    random_state=42,
    passes=100,
    iterations=200,
)
lda.print_topics(num_words=15)

[(0,
  '0.032*"grünen" + 0.026*"zitat" + 0.015*"weltkrieg" + 0.015*"grüne" + 0.015*"wünschte" + 0.014*"jugend" + 0.014*"miene" + 0.014*"zerbombt" + 0.014*"existenzberechtigung" + 0.014*"waziri" + 0.014*"land" + 0.011*"deutschland" + 0.011*"baerbock" + 0.011*"annalena" + 0.010*"platz"'),
 (1,
  '0.018*"bezeichnen" + 0.017*"spiel" + 0.017*"freunden" + 0.016*"idioten" + 0.016*"privat" + 0.016*"eingehalten" + 0.016*"heutzutage" + 0.016*"gespielt" + 0.016*"eigenes" + 0.016*"kleinen" + 0.015*"gesetzt" + 0.015*"couch" + 0.015*"maskenpflicht" + 0.015*"geltenden" + 0.015*"abstandsregeln"'),
 (2,
  '0.006*"menschen" + 0.003*"klimawandel" + 0.003*"auto" + 0.003*"klima" + 0.003*"china" + 0.003*"erde" + 0.002*"lügen" + 0.002*"strom" + 0.002*"verhindern" + 0.002*"beleidigen" + 0.002*"unterstellen" + 0.002*"normalen" + 0.002*"mensch" + 0.002*"gegeneinander" + 0.002*"spalten"'),
 (3,
  '0.009*"birne" + 0.008*"faschist" + 0.008*"halte" + 0.007*"martha" + 0.007*"liebe" + 0.007*"lügen" + 0.007*"schrei" +

<IPython.core.display.Javascript object>

In [18]:
lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=15,
    random_state=42,
    passes=50,
    iterations=100,
)
lda.print_topics(num_words=15)

[(0,
  '0.017*"google" + 0.015*"nazi" + 0.014*"aussage" + 0.014*"frau" + 0.014*"nachvollziehbare" + 0.014*"reaktion" + 0.013*"hält" + 0.013*"verbrechen" + 0.013*"gerechtfertigt" + 0.013*"bomben" + 0.013*"unschuldige" + 0.013*"maniac" + 0.012*"bürger" + 0.012*"kinder" + 0.012*"zitat"'),
 (1,
  '0.022*"afd" + 0.009*"partei" + 0.008*"corona" + 0.007*"glück" + 0.007*"altparteien" + 0.007*"leugnen" + 0.007*"durchgesetzt" + 0.007*"nazi" + 0.006*"lachten" + 0.006*"ideologie" + 0.006*"forderte" + 0.006*"intelligent" + 0.006*"verschärfte" + 0.005*"nebenbei" + 0.005*"leugnet"'),
 (2,
  '0.007*"schwachsinn" + 0.007*"sonne" + 0.005*"studien" + 0.005*"wärmer" + 0.004*"meinte" + 0.004*"diesel" + 0.004*"mitte" + 0.004*"behauptung" + 0.004*"behaupten" + 0.004*"glauben" + 0.004*"versteht" + 0.004*"beweisen" + 0.004*"kontext" + 0.004*"faz" + 0.004*"blödsinn"'),
 (3,
  '0.014*"eike" + 0.012*"mario" + 0.009*"account" + 0.009*"volt" + 0.008*"mehreren" + 0.008*"lese" + 0.008*"kanal" + 0.008*"afd" + 0.007*"t

<IPython.core.display.Javascript object>

### outputs
#### 1.
lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=10,
    random_state=100,
    chunksize=10000,
    passes=50,
    iterations=100,
)

 (0,
  '0.030*"gespielt" + 0.030*"privat" + 0.029*"maskenpflicht" + 0.029*"couch" + 0.029*"eingehalten" + 0.029*"geltenden" + 0.029*"abstandsregeln" + 0.029*"coronaregeln" + 0.029*"bezeichnen" + 0.029*"freunden"'),
 \
 (1,
  '0.036*"zitat" + 0.022*"zerbombt" + 0.022*"miene" + 0.022*"existenzberechtigung" + 0.022*"waziri" + 0.022*"weltkrieg" + 0.022*"wünschte" + 0.021*"grünen" + 0.021*"jugend" + 0.020*"grüne"'),
 \
 (2,
  '0.004*"klimawandel" + 0.004*"deutschland" + 0.004*"klima" + 0.004*"erde" + 0.003*"strom" + 0.003*"auto" + 0.003*"china" + 0.003*"menschen" + 0.003*"usa" + 0.003*"energie"'),
  \
  (3,
  '0.021*"lügen" + 0.019*"faschisten" + 0.016*"volksverpetzer" + 0.014*"lo" + 0.013*"hetze" + 0.012*"cordes" + 0.012*"axel" + 0.012*"widerliche" + 0.012*"stefan" + 0.011*"heute"'),
  \
  (4,
  '0.025*"unterstellen" + 0.025*"subjekte" + 0.024*"verhindern" + 0.024*"spalten" + 0.024*"gegeneinander" + 0.024*"meinungsaustausch" + 0.023*"normalen" + 0.023*"fiesen" + 0.023*"einheitliches" + 0.023*"erkennungsmerkmal"'),
  \
  (5,
  '0.002*"tube" + 0.002*"wählst" + 0.002*"fischer" + 0.002*"faschismus" + 0.002*"ströer" + 0.002*"programm" + 0.002*"social" + 0.002*"demokratische" + 0.002*"satire" + 0.002*"inhalte"'),
  \
  (6,
  '0.008*"video" + 0.006*"cdu" + 0.004*"rezo" + 0.004*"wählen" + 0.004*"gut" + 0.004*"menschen" + 0.004*"partei" + 0.004*"politik" + 0.004*"afd" + 0.003*"leute"'),
  \
  (7,
  '0.010*"reich" + 0.010*"afd" + 0.010*"eike" + 0.007*"kommunismus" + 0.007*"armer" + 0.006*"neidisch" + 0.006*"möchtest" + 0.006*"reiche" + 0.006*"troll" + 0.006*"gehört"'),
  \
  (8,
  '0.021*"grünen" + 0.013*"baerbock" + 0.013*"annalena" + 0.013*"platz" + 0.012*"saarland" + 0.012*"gewählte" + 0.012*"offiziell" + 0.012*"spitzenkandidat" + 0.012*"räumt" + 0.012*"männerfeindlichen"'),
  \
  (9,
  '0.025*"user" + 0.022*"hindern" + 0.021*"andersdenkenden" + 0.021*"frei" + 0.021*"verkommen" + 0.021*"widerlich" + 0.021*"ekelhaften" + 0.021*"lasst" + 0.020*"liebe" + 0.019*"fanboysgirls"')
  
#### 2.

lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=20,
    random_state=100,
    passes=100,
    iterations=200,
)

(0,
  '0.010*"rechten" + 0.010*"unterwegs" + 0.008*"kapitalismus" + 0.008*"glauben" + 0.007*"studie" + 0.007*"programm" + 0.007*"frieden" + 0.007*"afdwähler" + 0.006*"haha" + 0.006*"nato"'),
 \
 (1,
  '0.053*"zitat" + 0.038*"grünen" + 0.028*"volkstod" + 0.028*"geistige" + 0.028*"sebastian" + 0.028*"zuwanderung" + 0.028*"striegel" + 0.027*"linker" + 0.026*"brandstifter" + 0.026*"terror"'),
 \
 (2,
  '0.007*"deutschland" + 0.006*"grünen" + 0.006*"cdu" + 0.006*"menschen" + 0.005*"partei" + 0.005*"video" + 0.005*"afd" + 0.005*"klimawandel" + 0.005*"jahren" + 0.004*"welt"'),
 \
 (3,
  '0.022*"volksverpetzer" + 0.016*"andersdenkende" + 0.015*"netz" + 0.014*"fakenews" + 0.014*"hetze" + 0.013*"diffamieren" + 0.012*"meinungsfreiheit" + 0.012*"denunzieren" + 0.012*"üben" + 0.011*"widerliche"'),
 \
 (4,
  '0.028*"gegeneinander" + 0.027*"spalten" + 0.027*"unterstellen" + 0.026*"meinungsaustausch" + 0.026*"normalen" + 0.026*"einheitliches" + 0.026*"fiesen" + 0.026*"erkennungsmerkmal" + 0.026*"aufwiegeln" + 0.026*"festgesetzt"'),
 \
 (5,
  '0.023*"vogel" + 0.023*"methoden" + 0.023*"werner" + 0.022*"faschistischen" + 0.022*"aufhören" + 0.022*"diffamierungen" + 0.021*"zuspammen" + 0.021*"massiven" + 0.021*"kommentarfunktion" + 0.021*"legion"'),
 \
 (6,
  '0.035*"afd" + 0.020*"corona" + 0.020*"glück" + 0.020*"altparteien" + 0.018*"durchgesetzt" + 0.018*"partei" + 0.017*"lachten" + 0.016*"forderte" + 0.016*"verschärfte" + 0.014*"behauptungen"'),
 \
 (7,
  '0.008*"reich" + 0.006*"oh" + 0.006*"staat" + 0.005*"kommunismus" + 0.005*"möchtest" + 0.005*"gehört" + 0.005*"leugnen" + 0.005*"halt" + 0.005*"kosten" + 0.005*"impressum"'),
 \
 (8,
  '0.008*"darum" + 0.008*"ströer" + 0.008*"herr" + 0.007*"schreiben" + 0.007*"überzeugen" + 0.007*"alter" + 0.006*"regiert" + 0.006*"rentner" + 0.006*"verändern" + 0.005*"angeschaut"'),
 \
 (9,
  '0.032*"hindern" + 0.032*"andersdenkenden" + 0.032*"verkommen" + 0.032*"widerlich" + 0.031*"frei" + 0.031*"ekelhaften" + 0.031*"user" + 0.030*"lasst" + 0.029*"besonders" + 0.028*"absolut"'),
 \
 (10,
  '0.010*"schwachsinn" + 0.008*"rechts" + 0.007*"schreibst" + 0.007*"beide" + 0.007*"nich" + 0.006*"gott" + 0.006*"blöd" + 0.006*"jemanden" + 0.006*"runter" + 0.005*"antworten"'),
 \
 (11,
  '0.014*"video" + 0.008*"cdu" + 0.008*"rezo" + 0.007*"wählen" + 0.007*"gut" + 0.006*"meinung" + 0.005*"leute" + 0.005*"menschen" + 0.005*"afd" + 0.005*"politik"'),
 \
 (12,
  '0.019*"denk" + 0.016*"mario" + 0.013*"benutzen" + 0.013*"vernünftige" + 0.011*"tierschutzpartei" + 0.010*"brauchst" + 0.010*"faschismus" + 0.010*"gelöscht" + 0.010*"töten" + 0.010*"begründung"'),
 \
 (13,
  '0.040*"grünen" + 0.032*"baerbock" + 0.031*"annalena" + 0.027*"saarland" + 0.026*"gewählte" + 0.026*"offiziell" + 0.026*"spitzenkandidat" + 0.026*"räumt" + 0.026*"männerfeindlichen" + 0.026*"antidemokratischen"'),
 \
 (14,
  '0.007*"wissenschaftler" + 0.007*"sonne" + 0.005*"wissenschaft" + 0.005*"grün" + 0.005*"janich" + 0.005*"ach" + 0.005*"wärmer" + 0.005*"menschengemachten" + 0.005*"flüchtlinge" + 0.005*"satz"'),
 \
 (15,
  '0.012*"meinst" + 0.011*"glaubst" + 0.007*"beiden" + 0.006*"kontext" + 0.006*"glaub" + 0.006*"mitte" + 0.006*"vollkommen" + 0.005*"kennst" + 0.005*"typen" + 0.005*"manipulation"'),
 \
 (16,
  '0.018*"nazi" + 0.014*"aussage" + 0.014*"gerechtfertigt" + 0.014*"frau" + 0.014*"unschuldige" + 0.013*"bomben" + 0.013*"reaktion" + 0.013*"nachvollziehbare" + 0.013*"hält" + 0.013*"verbrechen"'),
 \
 (17,
  '0.034*"lügen" + 0.033*"lo" + 0.028*"faschisten" + 0.024*"unterstellungen" + 0.023*"irma" + 0.023*"achtung" + 0.022*"beleidigungen" + 0.020*"user" + 0.020*"heute" + 0.019*"gespammt"'),
 \
 (18,
  '0.011*"komm" + 0.010*"wählst" + 0.010*"faz" + 0.009*"schwarz" + 0.008*"leiden" + 0.007*"halbwahrheiten" + 0.007*"irgendetwas" + 0.007*"begründet" + 0.007*"einsetzt" + 0.007*"volkes"'),
 \
 (19,
  '0.033*"privat" + 0.033*"gespielt" + 0.033*"couch" + 0.033*"geltenden" + 0.033*"eingehalten" + 0.033*"bezeichnen" + 0.033*"maskenpflicht" + 0.033*"abstandsregeln" + 0.033*"coronaregeln" + 0.033*"freunden"')
  
#### 3.  

lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=7,
    random_state=42,
    passes=100,
    iterations=200,
)

(0,
  '0.032*"grünen" + 0.026*"zitat" + 0.015*"weltkrieg" + 0.015*"grüne" + 0.015*"wünschte" + 0.014*"jugend" + 0.014*"miene" + 0.014*"zerbombt" + 0.014*"existenzberechtigung" + 0.014*"waziri" + 0.014*"land" + 0.011*"deutschland" + 0.011*"baerbock" + 0.011*"annalena" + 0.010*"platz"'),
 \
 (1,
  '0.018*"bezeichnen" + 0.017*"spiel" + 0.017*"freunden" + 0.016*"idioten" + 0.016*"privat" + 0.016*"eingehalten" + 0.016*"heutzutage" + 0.016*"gespielt" + 0.016*"eigenes" + 0.016*"kleinen" + 0.015*"gesetzt" + 0.015*"couch" + 0.015*"maskenpflicht" + 0.015*"geltenden" + 0.015*"abstandsregeln"'),
 \
 (2,
  '0.006*"menschen" + 0.003*"klimawandel" + 0.003*"auto" + 0.003*"klima" + 0.003*"china" + 0.003*"erde" + 0.002*"lügen" + 0.002*"strom" + 0.002*"verhindern" + 0.002*"beleidigen" + 0.002*"unterstellen" + 0.002*"normalen" + 0.002*"mensch" + 0.002*"gegeneinander" + 0.002*"spalten"'),
 \
 (3,
  '0.009*"birne" + 0.008*"faschist" + 0.008*"halte" + 0.007*"martha" + 0.007*"liebe" + 0.007*"lügen" + 0.007*"schrei" + 0.007*"stummer" + 0.006*"pfahl" + 0.006*"verbreitet" + 0.006*"hauptsache" + 0.005*"versager" + 0.005*"beleidigungen" + 0.005*"leider" + 0.005*"stefan"'),
 \
 (4,
  '0.007*"video" + 0.006*"cdu" + 0.004*"gut" + 0.004*"rezo" + 0.004*"wählen" + 0.004*"partei" + 0.004*"grünen" + 0.004*"meinung" + 0.003*"politik" + 0.003*"parteien" + 0.003*"afd" + 0.003*"deutschland" + 0.003*"leute" + 0.003*"wirklich" + 0.003*"menschen"'),
 \
 (5,
  '0.018*"user" + 0.017*"faschisten" + 0.017*"axel" + 0.017*"lügen" + 0.017*"cordes" + 0.016*"maniac" + 0.015*"irma" + 0.015*"lo" + 0.014*"beleidigen" + 0.014*"unterstellungen" + 0.014*"beleidigungen" + 0.013*"stefan" + 0.013*"fanboysgirls" + 0.012*"lena" + 0.012*"anna"'),
 \
 (6,
  '0.007*"afd" + 0.004*"reich" + 0.004*"rechts" + 0.004*"wählt" + 0.003*"gehört" + 0.003*"halt" + 0.003*"google" + 0.003*"wissenschaftlichen" + 0.003*"möchtest" + 0.003*"staat" + 0.002*"hand" + 0.002*"eike" + 0.002*"kommunismus" + 0.002*"rezo" + 0.002*"leute"')
  
#### 4.

lda = models.LdaModel(
    corpus=corpus_sparse,
    id2word=id2word,
    num_topics=15,
    random_state=42,
    passes=50,
    iterations=100,
)

(0,
  '0.017*"google" + 0.015*"nazi" + 0.014*"aussage" + 0.014*"frau" + 0.014*"nachvollziehbare" + 0.014*"reaktion" + 0.013*"hält" + 0.013*"verbrechen" + 0.013*"gerechtfertigt" + 0.013*"bomben" + 0.013*"unschuldige" + 0.013*"maniac" + 0.012*"bürger" + 0.012*"kinder" + 0.012*"zitat"'),
 \
 (1,
  '0.022*"afd" + 0.009*"partei" + 0.008*"corona" + 0.007*"glück" + 0.007*"altparteien" + 0.007*"leugnen" + 0.007*"durchgesetzt" + 0.007*"nazi" + 0.006*"lachten" + 0.006*"ideologie" + 0.006*"forderte" + 0.006*"intelligent" + 0.006*"verschärfte" + 0.005*"nebenbei" + 0.005*"leugnet"'),
 \
 (2,
  '0.007*"schwachsinn" + 0.007*"sonne" + 0.005*"studien" + 0.005*"wärmer" + 0.004*"meinte" + 0.004*"diesel" + 0.004*"mitte" + 0.004*"behauptung" + 0.004*"behaupten" + 0.004*"glauben" + 0.004*"versteht" + 0.004*"beweisen" + 0.004*"kontext" + 0.004*"faz" + 0.004*"blödsinn"'),
 \
 (3,
  '0.014*"eike" + 0.012*"mario" + 0.009*"account" + 0.009*"volt" + 0.008*"mehreren" + 0.008*"lese" + 0.008*"kanal" + 0.008*"afd" + 0.007*"troll" + 0.007*"tubeone" + 0.006*"löschen" + 0.006*"versager" + 0.006*"boomer" + 0.006*"youtube" + 0.006*"kommentare"'),
 \
 (4,
  '0.009*"video" + 0.007*"cdu" + 0.006*"menschen" + 0.006*"rezo" + 0.005*"grünen" + 0.005*"gut" + 0.005*"wählen" + 0.005*"partei" + 0.005*"deutschland" + 0.005*"afd" + 0.005*"meinung" + 0.004*"politik" + 0.004*"leute" + 0.004*"parteien" + 0.004*"klimawandel"'),
 \
 (5,
  '0.017*"beleidigungen" + 0.017*"vogel" + 0.017*"faschistischen" + 0.017*"werner" + 0.017*"diffamierungen" + 0.016*"massiven" + 0.016*"methoden" + 0.016*"kommentarfunktion" + 0.016*"unterstellungen" + 0.016*"zuspammen" + 0.016*"aufhören" + 0.016*"usern" + 0.016*"legion" + 0.016*"bedanken" + 0.016*"aufgrund"'),
 \
 (6,
  '0.006*"klimaforscher" + 0.005*"wählst" + 0.005*"kannst" + 0.005*"sachlich" + 0.005*"komisch" + 0.004*"grenzen" + 0.004*"korrekt" + 0.004*"teilen" + 0.004*"gehirn" + 0.004*"runter" + 0.004*"spinner" + 0.004*"kennst" + 0.004*"jedenfalls" + 0.004*"unten" + 0.004*"spielt"'),
 \
 (7,
  '0.006*"kannst" + 0.006*"darf" + 0.006*"rezos" + 0.006*"schau" + 0.005*"dumm" + 0.004*"oh" + 0.004*"ströer" + 0.004*"grün" + 0.004*"alter" + 0.004*"einstellung" + 0.004*"werbung" + 0.004*"funktioniert" + 0.004*"eigene" + 0.004*"herr" + 0.004*"wichtig"'),
 \
 (8,
  '0.042*"zitat" + 0.031*"grünen" + 0.023*"volkstod" + 0.022*"sebastian" + 0.022*"zuwanderung" + 0.022*"striegel" + 0.021*"brandstifter" + 0.020*"linker" + 0.020*"geistige" + 0.020*"zerbombt" + 0.020*"miene" + 0.020*"existenzberechtigung" + 0.020*"weltkrieg" + 0.020*"waziri" + 0.020*"terror"'),
 \
 (9,
  '0.023*"beleidigen" + 0.022*"spalten" + 0.022*"gegeneinander" + 0.021*"lügen" + 0.021*"unterstellen" + 0.021*"einheitliches" + 0.020*"meinungsaustausch" + 0.020*"fiesen" + 0.020*"erkennungsmerkmal" + 0.020*"aufwiegeln" + 0.020*"festgesetzt" + 0.020*"linkspopulisten" + 0.020*"rezofanboys" + 0.020*"normalen" + 0.020*"subjekte"'),
 \
 (10,
  '0.028*"gespielt" + 0.028*"eingehalten" + 0.028*"couch" + 0.028*"maskenpflicht" + 0.028*"spiel" + 0.028*"privat" + 0.028*"geltenden" + 0.028*"abstandsregeln" + 0.028*"coronaregeln" + 0.028*"bezeichnen" + 0.028*"freunden" + 0.027*"gesetzt" + 0.027*"heutzutage" + 0.027*"idioten" + 0.027*"eigenes"'),
 \
 (11,
  '0.030*"grünen" + 0.023*"baerbock" + 0.023*"annalena" + 0.019*"gewählte" + 0.018*"platz" + 0.018*"saarland" + 0.018*"offiziell" + 0.018*"spitzenkandidat" + 0.018*"räumt" + 0.018*"männerfeindlichen" + 0.018*"antidemokratischen" + 0.018*"antidemokratisch" + 0.017*"männerfeindlich" + 0.016*"fordern" + 0.015*"weiterhin"'),
 \
 (12,
  '0.026*"user" + 0.025*"lo" + 0.024*"irma" + 0.023*"liebe" + 0.021*"faschisten" + 0.019*"lügen" + 0.018*"faschist" + 0.017*"beleidigen" + 0.016*"widerlich" + 0.016*"andersdenkenden" + 0.016*"hindern" + 0.016*"verkommen" + 0.016*"ekelhaften" + 0.016*"frei" + 0.015*"besonders"'),
 \
 (13,
  '0.009*"meinst" + 0.008*"anscheinend" + 0.007*"denkst" + 0.007*"bilden" + 0.007*"willst" + 0.007*"kannst" + 0.006*"widerlegen" + 0.006*"wählt" + 0.006*"kritisieren" + 0.005*"steckt" + 0.005*"trump" + 0.005*"beeinflussen" + 0.005*"erwarten" + 0.004*"gehst" + 0.004*"selbstverständlich"'),
 \
 (14,
  '0.007*"china" + 0.006*"bezahlt" + 0.004*"kannst" + 0.004*"trend" + 0.004*"co²" + 0.004*"arbeitsplätze" + 0.004*"arbeitet" + 0.004*"wissenschaftlern" + 0.004*"öl" + 0.004*"irgendwas" + 0.004*"planeten" + 0.003*"internet" + 0.003*"familie" + 0.003*"darfst" + 0.003*"typen"')